**CCXT L2 Order Book**

In [5]:
import ccxt
import pandas as pd
from datetime import datetime, timezone

# Top 10 cryptocurrencies by market cap (USDT pairs on Binance)
TEN_LARGEST_SYMBOLS = [
    "BTC/USDT", "ETH/USDT", "BNB/USDT", "SOL/USDT", "XRP/USDT",
    "USDC/USDT", "DOGE/USDT", "ADA/USDT", "AVAX/USDT", "DOT/USDT",
]

# Optional: use another exchange, e.g. ccxt.kraken(), ccxt.coinbase()
exchange = ccxt.binance({"enableRateLimit": True})

# Fetch L2 order book for each symbol (limit = depth; None = exchange default, often 20–100)
LIMIT = 50  # number of bid/ask levels per side

l2_ccxt = {}
for symbol in TEN_LARGEST_SYMBOLS:
    try:
        ob = exchange.fetch_order_book(symbol, limit=LIMIT)
        # CCXT returns bids/asks as [[price, size], ...]
        bids = pd.DataFrame(ob["bids"], columns=["price", "size"])
        asks = pd.DataFrame(ob["asks"], columns=["price", "size"])
        bids["level"] = range(1, len(bids) + 1)
        asks["level"] = range(1, len(asks) + 1)
        l2_ccxt[symbol] = {
            "timestamp": ob.get("timestamp") and datetime.fromtimestamp(ob["timestamp"] / 1000, tz=timezone.utc) or datetime.now(timezone.utc),
            "bids": bids[["level", "price", "size"]],
            "asks": asks[["level", "price", "size"]],
        }
    except Exception as e:
        print(f"Skip {symbol}: {e}")

# Example: show L2 for one symbol
sym = "ETH/USDT"
if sym in l2_ccxt:
    d = l2_ccxt[sym]
    print(f"L2 {sym} @ {d['timestamp']}")
    display(d["bids"].head(10))
    display(d["asks"].head(10))

L2 ETH/USDT @ 2026-02-21 08:56:54.273169+00:00


,level,price,size
0,1,1967.43,47.6837
1,2,1967.42,8.2090
2,3,1967.41,0.0079
3,4,1967.40,0.1401
4,5,1967.39,1.9394
5,6,1967.38,6.3695
6,7,1967.37,0.0053
7,8,1967.36,0.0579
8,9,1967.35,0.0053
9,10,1967.34,1.2717


,level,price,size
0,1,1967.44,8.1034
1,2,1967.45,0.0053
2,3,1967.46,0.0053
3,4,1967.47,0.0131
4,5,1967.48,0.0027
5,6,1967.49,0.0109
6,7,1967.50,0.0079
7,8,1967.51,0.0308
8,9,1967.52,0.0307
9,10,1967.53,0.0105


In [4]:
# L2 summary across all symbols (ccxt)
summary = []
for symbol, d in l2_ccxt.items():
    b, a = d["bids"], d["asks"]
    summary.append({
        "symbol": symbol,
        "bid_levels": len(b),
        "ask_levels": len(a),
        "best_bid": b["price"].iloc[0] if len(b) else None,
        "best_ask": a["price"].iloc[0] if len(a) else None,
        "bid_size_total": b["size"].sum(),
        "ask_size_total": a["size"].sum(),
    })
pd.DataFrame(summary)

,symbol,bid_levels,ask_levels,best_bid,best_ask,bid_size_total,ask_size_total
0,BTC/USDT,50,50,68032.34000,68032.35000,8.853490e+00,1.439830e+00
1,ETH/USDT,50,50,1967.66000,1967.67000,2.342961e+02,2.186148e+02
2,BNB/USDT,50,50,627.49000,627.50000,3.857610e+02,3.669480e+02
3,SOL/USDT,50,50,84.67000,84.68000,4.167448e+04,4.892526e+04
4,XRP/USDT,50,50,1.43380,1.43390,1.745771e+06,1.597222e+06
5,USDC/USDT,50,50,1.00020,1.00030,1.161319e+08,1.246828e+08
6,DOGE/USDT,50,50,0.10025,0.10026,1.664539e+07,1.408994e+07
7,ADA/USDT,50,50,0.28340,0.28350,4.139438e+06,3.140890e+06
8,AVAX/USDT,50,50,9.19000,9.20000,1.487183e+05,9.523798e+04
9,DOT/USDT,50,50,1.36800,1.36900,4.102736e+05,1.951149e+05
